In [44]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'omkar_proj.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [76]:
from account.models import (
    Owner
)
from warehouse.models import (
    Company,
    Gala
)
from contract.models import (
    Farmer as ContractFarmer
)
from django.db.models import Case, When, OuterRef,Count,Value,CharField

In [117]:
live_and_license_warehouse_qs = Company.objects.get(
                name="Omkar"
            ).get_properties.all().values(
                "uid", "property_name", "city","is_allotted_to_farmer"
            ).annotate(
                total_gala_count = 
                Case(
                    When(
                        is_allotted_to_farmer = True,
                        then=Gala.objects.filter(
                            warehouse__uid=OuterRef("uid"),is_allotted_to_farmer=True).values(
                            "warehouse__uid"
                    ).annotate(
                        total_gala_count=Count("id")).values(
                            "total_gala_count"
                    )),
                    When(
                        is_allotted_to_farmer = False,
                        then=Gala.objects.filter(
                            warehouse__uid=OuterRef("uid")).values(
                            "warehouse__uid"
                    ).annotate(
                        total_gala_count=Count("id")).values(
                            "total_gala_count"
                    ))
                ),
                total_allotted_galas = Case(
                    When(
                        is_allotted_to_farmer = True,
                        then=Gala.objects.filter(
                            warehouse__uid=OuterRef("uid"),is_allotted_to_farmer=True,is_allotted_to_rental=True).values(
                            "warehouse__uid"
                    ).annotate(
                        total_gala_count=Count("id")).values(
                            "total_gala_count"
                    )),
                    When(
                        is_allotted_to_farmer = False,
                        then=Gala.objects.filter(
                            warehouse__uid=OuterRef("uid"),is_allotted_to_rental=True).values(
                            "warehouse__uid"
                    ).annotate(
                        total_gala_count=Count("id")).values(
                            "total_gala_count"
                    ))
                ),
                total_remaining_galas = Case(
                    When(
                        is_allotted_to_farmer = True,
                        then=Gala.objects.filter(
                            warehouse__uid=OuterRef("uid"),is_allotted_to_farmer=True,is_allotted_to_rental=False).values(
                            "warehouse__uid"
                    ).annotate(
                        total_gala_count=Count("id")).values(
                            "total_gala_count"
                    )),
                    When(
                        is_allotted_to_farmer = False,
                        then=Gala.objects.filter(
                            warehouse__uid=OuterRef("uid"),is_allotted_to_rental=False).values(
                            "warehouse__uid"
                    ).annotate(
                        total_gala_count=Count("id")).values(
                            "total_gala_count"
                    ))
                ),
                owner_type = Case(
                    When(
                        is_allotted_to_farmer = False,
                        then=Value(Owner.objects.first().username)),
                    When(
                        is_allotted_to_farmer = True,
                        then=ContractFarmer.objects.filter(warehouse__uid = OuterRef("uid")).values("user__username")),output_field = CharField()
                    
                ),
                
            )

In [119]:
live_and_license_warehouse_qs

<QuerySet [{'uid': UUID('e99c4212-8f6e-4bea-9efd-29518286c30e'), 'property_name': 'Omkar Warehouse 5', 'city': '...', 'is_allotted_to_farmer': False, 'total_gala_count': 2, 'total_allotted_galas': 1, 'total_remaining_galas': 1, 'owner_type': 'admin'}, {'uid': UUID('77d87c6a-afc4-4431-8930-cc92b360a975'), 'property_name': 'Omkar Warehouse 4', 'city': '...', 'is_allotted_to_farmer': False, 'total_gala_count': None, 'total_allotted_galas': None, 'total_remaining_galas': None, 'owner_type': 'admin'}, {'uid': UUID('e5abcbd1-1e6e-46d7-ab28-ea2bbf39e4bd'), 'property_name': 'Flip Kart', 'city': 'Indore', 'is_allotted_to_farmer': False, 'total_gala_count': 2, 'total_allotted_galas': 1, 'total_remaining_galas': 1, 'owner_type': 'admin'}, {'uid': UUID('0d3c64ce-79a8-4ed9-8634-c0e0a78a07cc'), 'property_name': 'Flip Kart', 'city': 'Indore', 'is_allotted_to_farmer': False, 'total_gala_count': None, 'total_allotted_galas': None, 'total_remaining_galas': None, 'owner_type': 'admin'}, {'uid': UUID('b33

In [102]:
ContractFarmer.objects.first().polymorphic_ctype.name

'farmer'